In [2]:
# Collecting job list from google job search

# Importing libraries
import os
import re
import pandas as pd
import gspread

from oauth2client.service_account import ServiceAccountCredentials
from time import sleep
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException
from selenium.webdriver.support import expected_conditions as EC
from timeit import default_timer as timer

job_results = []

In [59]:
def setup_selenium():

    # Configure selenium
    options = webdriver.ChromeOptions()
    # Set user data directory to avoid logging in every time
    options.add_argument(f"user-data-dir=E:\\Repository\\data-jobs-in-indonesia\\Browser data")

    driver = webdriver.Chrome(options=options)

    return driver

def load_all_jobs(driver, left_pane):
    # Scroll to the bottom of the list until all jobs are loaded
    all_jobs_loaded = False
    while not all_jobs_loaded:

        # Get old scroll height
        previous_scroll_height = driver.execute_script("return arguments[0].scrollHeight;", left_pane)

        # Scroll to bottom of the list (fetches more jobs)
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", left_pane)

        # Grant some time for messages to load
        sleep(1)

        # Get current scroll height
        current_scroll_height = driver.execute_script("return arguments[0].scrollHeight;", left_pane)
        # Check if all messages were loaded by comparing the scroll heights
        if current_scroll_height == previous_scroll_height:
            # Scroll back to top of the list to start collecting jobs
            driver.execute_script("arguments[0].scrollTop = 0", left_pane)
            all_jobs_loaded = True

def get_jobs(driver, left_pane):
    # Looping through jobs list
    for job in left_pane.find_elements(By.TAG_NAME, 'li'):
        # Clicking on the job
        job.click()

        # Getting job details
        # Extract details information
        title = job.find_element(By.CLASS_NAME, 'BjJfJf').text
        company = job.find_element(By.CLASS_NAME, 'vNEEBe').text
        location = job.find_element(By.CLASS_NAME, 'Qk80Jf').text
        # Extract details after via based on the value it contains
        details = job.find_element(By.CLASS_NAME, 'PwjeAc').text.split('\n')
        via = ''
        date_posted = ''
        salary = ''
        job_type = ''
        for i in details:
            if 'melalui' in i:
                via = i
            elif 'yang' in i:
                date_posted = i
            elif 'Rp' in i:
                salary = i
            else:
                job_type = i

        # Getting job description from right pane
        description_pane = driver.find_element(By.CLASS_NAME, 'whazf')
        # Try if description is expandable
        try:
            # Expand the description
            description_pane.find_element(By.CLASS_NAME, 'mjkhcd').click()
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'config-text-expandable')))
            description = description_pane.find_element(By.CLASS_NAME, 'config-text-expandable').text
        except NoSuchElementException:
            description = description_pane.find_element(By.CLASS_NAME, 'HBvzbc').text

        # Append to job_results
        job_results.append([title, company, location, via, date_posted, salary, job_type, description])

def convert_salary(text):
    text = text.replace('Rp ', '').replace('.', '').replace(' rb', '000').replace('per bulan', '')
    if re.search(r'(\d),(\d{2}) jt', text):
        text = text.replace(' jt', '0000')
    elif re.search(r'(\d),(\d) jt', text):
        text = text.replace(' jt', '00000')
    else:
        text = text.replace(' jt', '000000')

    return int(text.replace(',', ''))

In [3]:
# Start the script
job_results.clear()
driver = setup_selenium()
job_search = ['data+analyst', 'data+scientist', 'data+engineer']

# Loop Through Job Search
for job in job_search:
    driver.get(f'https://www.google.com/search?q={job}&ibp=htl;jobs#htivrt=jobs&fpstate=tldetail&htilrad=-1.0&htidocid')

    # Wait for the page to load
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, 'li')))
    left_pane = driver.find_element(By.CLASS_NAME, 'zxU94d')
    load_all_jobs(driver, left_pane)
    get_jobs(driver, left_pane)

# Close the browser
driver.quit()

# Create dataframe
df = pd.DataFrame(job_results, columns=['title', 'company', 'location', 'via', 'date_posted', 'salary', 'job_type', 'description'])
df = df.drop_duplicates(subset=['title', 'company', 'location', 'via', 'salary'])
df.to_csv('job_list.csv', index=False)

In [79]:
# Reset dataframe
df = pd.DataFrame(job_results, columns=['title', 'company', 'location', 'via', 'date_posted', 'salary', 'job_type', 'description'])
df.head()

,title,company,location,via,date_posted,date_searched,salary,job_type,description
0,Junior Data Analyst,Jiva,"Makassar, Kota Makassar, Sulawesi Selatan",melalui LinkedIn,4 hari yang lalu,2023-11-05,,Pekerjaan tetap,General Overview\n\nEvaluate and analyze poten...
1,Data Analyst,Koperasi Simpan Pinjam Sinergi Dana Cipta,"Makassar, Kota Makassar, Sulawesi Selatan",melalui Trabajo.org,3 hari yang lalu,2023-11-05,,Pekerjaan tetap,Persyaratan\n\nAssociate Degree (D3)\n\n1 Year...
2,Data Analyst,PT Tokopedia,"Jakarta, Daerah Khusus Ibukota Jakarta",melalui LinkedIn,2 hari yang lalu,2023-11-05,,Pekerjaan tetap,What You Will Do\n• Become SME (Subject Matter...
3,Data Analyst,Kinarya Alihdaya Mandiri PT,"Jakarta, Daerah Khusus Ibukota Jakarta",melalui Trabajo.org,24 jam yang lalu,2023-11-05,,Pekerjaan tetap,Pendidikan Minimal D3 Teknik Informatika/Siste...
4,Data Analyst,Traveloka,"Jakarta, Daerah Khusus Ibukota Jakarta",melalui Cakap Career,5 hari yang lalu,2023-11-05,,Pekerjaan tetap,As a Data Analyst should develop dashboards no...


In [80]:
# Clean and Standardize Data
# Convert date_posted to correct dd/mm/yyyy format
df['parse_date_posted'] = df['date_posted'].apply(lambda x: datetime.today().date() - timedelta(days=int(x.split(' ')[0])) if x != '' and 'hari' in x else datetime.today().date() if x != '' and 'jam' in x else x)
# Remove 'melalu' from via column
df['posted_via'] = df['via'].str.replace('melalui', '')
# Remove '(+n lainnya)' from location column using regex
df['location'] = df['location'].apply(lambda x: re.sub(r' \(\+\d+ lainnya\)', '', x))
# Get the city from location column
df['city'] = df['location'].apply(lambda x: x.split(',')[0] if x != '' else x)
df['city'] = df['city'].str.replace('Kota ', '').str.replace('Kab. ', '').str.replace('Kabupaten ', '')
# Get the province from location column
df['province'] = df['location'].apply(lambda x: x.split(',')[-1].strip() if x != '' else x)
# Clean Salary
# Drop row if salary contain 'per hari' or 'per tahun'
df = df[~df['salary'].str.contains('per hari|per tahun')]
# Split salary into min_salary & max_salary
df['min_salary'] = df['salary'].apply(lambda x: convert_salary(x.split('–')[0]) if '–' in x else convert_salary(x) if x !='' else x)
df['max_salary'] = df['salary'].apply(lambda x: convert_salary(x.split('–')[1]) if '–' in x else convert_salary(x) if x !='' else x)

df.head()


,title,company,location,via,date_posted,date_searched,salary,job_type,description,parse_date_posted,posted_on,city,province,min_salary,max_salary
0,Junior Data Analyst,Jiva,"Makassar, Kota Makassar, Sulawesi Selatan",melalui LinkedIn,4 hari yang lalu,2023-11-05,,Pekerjaan tetap,General Overview\n\nEvaluate and analyze poten...,2023-11-01,LinkedIn,Makassar,Sulawesi Selatan,,
1,Data Analyst,Koperasi Simpan Pinjam Sinergi Dana Cipta,"Makassar, Kota Makassar, Sulawesi Selatan",melalui Trabajo.org,3 hari yang lalu,2023-11-05,,Pekerjaan tetap,Persyaratan\n\nAssociate Degree (D3)\n\n1 Year...,2023-11-02,Trabajo.org,Makassar,Sulawesi Selatan,,
2,Data Analyst,PT Tokopedia,"Jakarta, Daerah Khusus Ibukota Jakarta",melalui LinkedIn,2 hari yang lalu,2023-11-05,,Pekerjaan tetap,What You Will Do\n• Become SME (Subject Matter...,2023-11-03,LinkedIn,Jakarta,Daerah Khusus Ibukota Jakarta,,
3,Data Analyst,Kinarya Alihdaya Mandiri PT,"Jakarta, Daerah Khusus Ibukota Jakarta",melalui Trabajo.org,24 jam yang lalu,2023-11-05,,Pekerjaan tetap,Pendidikan Minimal D3 Teknik Informatika/Siste...,2023-11-05,Trabajo.org,Jakarta,Daerah Khusus Ibukota Jakarta,,
4,Data Analyst,Traveloka,"Jakarta, Daerah Khusus Ibukota Jakarta",melalui Cakap Career,5 hari yang lalu,2023-11-05,,Pekerjaan tetap,As a Data Analyst should develop dashboards no...,2023-10-31,Cakap Career,Jakarta,Daerah Khusus Ibukota Jakarta,,


In [3]:
df = pd.read_csv('E:/Repository/data-jobs-in-indonesia/job_list.csv')
df = df.drop_duplicates(subset=['title', 'company', 'location', 'via', 'salary'])
df.head()

,title,company,location,via,date_posted,date_searched,salary,job_type,description,parse_date_posted,posted_via,city,province,min_salary,max_salary
0,Junior Data Analyst,Jiva,"Makassar, Kota Makassar, Sulawesi Selatan",melalui LinkedIn,4 hari yang lalu,2023-11-05,NaN,Pekerjaan tetap,General Overview\n\nEvaluate and analyze poten...,2023-11-01,LinkedIn,Makassar,Sulawesi Selatan,NaN,NaN
1,Data Analyst,Koperasi Simpan Pinjam Sinergi Dana Cipta,"Makassar, Kota Makassar, Sulawesi Selatan",melalui Trabajo.org,3 hari yang lalu,2023-11-05,NaN,Pekerjaan tetap,Persyaratan\n\nAssociate Degree (D3)\n\n1 Year...,2023-11-02,Trabajo.org,Makassar,Sulawesi Selatan,NaN,NaN
2,Data Analyst,PT Tokopedia,"Jakarta, Daerah Khusus Ibukota Jakarta",melalui LinkedIn,2 hari yang lalu,2023-11-05,NaN,Pekerjaan tetap,What You Will Do\n• Become SME (Subject Matter...,2023-11-03,LinkedIn,Jakarta,Daerah Khusus Ibukota Jakarta,NaN,NaN
3,Data Analyst,Kinarya Alihdaya Mandiri PT,"Jakarta, Daerah Khusus Ibukota Jakarta",melalui Trabajo.org,24 jam yang lalu,2023-11-05,NaN,Pekerjaan tetap,Pendidikan Minimal D3 Teknik Informatika/Siste...,2023-11-05,Trabajo.org,Jakarta,Daerah Khusus Ibukota Jakarta,NaN,NaN
4,Data Analyst,Traveloka,"Jakarta, Daerah Khusus Ibukota Jakarta",melalui Cakap Career,5 hari yang lalu,2023-11-05,NaN,Pekerjaan tetap,As a Data Analyst should develop dashboards no...,2023-10-31,Cakap Career,Jakarta,Daerah Khusus Ibukota Jakarta,NaN,NaN


In [8]:
# Reorder the data
cols_order = ['parse_date_posted', 'title', 'company', 'city', 'province', 'min_salary', 'max_salary', 'posted_via', 'job_type', 'description']
final_df = df[cols_order]
final_df

,parse_date_posted,title,company,city,province,min_salary,max_salary,posted_via,job_type,description
0,2023-11-01,Junior Data Analyst,Jiva,Makassar,Sulawesi Selatan,,,LinkedIn,Pekerjaan tetap,General Overview\n\nEvaluate and analyze poten...
1,2023-11-02,Data Analyst,Koperasi Simpan Pinjam Sinergi Dana Cipta,Makassar,Sulawesi Selatan,,,Trabajo.org,Pekerjaan tetap,Persyaratan\n\nAssociate Degree (D3)\n\n1 Year...
2,2023-11-03,Data Analyst,PT Tokopedia,Jakarta,Daerah Khusus Ibukota Jakarta,,,LinkedIn,Pekerjaan tetap,What You Will Do\n• Become SME (Subject Matter...
3,2023-11-05,Data Analyst,Kinarya Alihdaya Mandiri PT,Jakarta,Daerah Khusus Ibukota Jakarta,,,Trabajo.org,Pekerjaan tetap,Pendidikan Minimal D3 Teknik Informatika/Siste...
4,2023-10-31,Data Analyst,Traveloka,Jakarta,Daerah Khusus Ibukota Jakarta,,,Cakap Career,Pekerjaan tetap,As a Data Analyst should develop dashboards no...
...,...,...,...,...,...,...,...,...,...,...
472,2023-11-01,Data Engineer,Kompas Gramedia,Jakarta,Daerah Khusus Ibukota Jakarta,,,BeBee,Pekerjaan tetap,Identify and develop a strategy on long term d...
473,,Data Engineer,NTT,Indonesia,Indonesia,,,NodeFlair,Pekerjaan tetap,"Working at NTT\n• Designs, builds and oversees..."
474,,Data Engineer - Data Warehouse,Traveloka,Jakarta,Daerah Khusus Ibukota Jakarta,,,LinkedIn,Pekerjaan tetap,It's fun to work in a company where people tru...
475,2023-11-02,Data Analyst,PT Jutarasa Abadi,Jakarta,Daerah Khusus Ibukota Jakarta,,,Trabajo.org,Pekerjaan tetap,Persyaratan\n\nBachelor’s degree from an accre...


In [11]:
# Connect to Google Sheets API and update worksheet
scope = ['https://www.googleapis.com/auth/spreadsheets',
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive"]
keyfile_path = os.path.join(os.getcwd(), 'credentials.json')
creds = ServiceAccountCredentials.from_json_keyfile_name(keyfile_path, scope)
client = gspread.authorize(creds)
spreadsheet = client.open('jobs_data')
worksheet = spreadsheet.worksheet('selenium_data')

# Convert DataFrame to a list of lists
data = [cols_order] + final_df.values.tolist()

# Append the data to the worksheet
worksheet.update(range_name='A1', values=data, raw=True)


C:\Users\refnh\AppData\Local\Temp\ipykernel_6632\2037205704.py:15: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update(range_name='A1', values=data, raw=True)


{'spreadsheetId': '1wcDWvUZqVh9oMtz_hXlrzW1KJQa-Co8T94paavnNgS8',
 'updatedRange': 'selenium_data!A1:J478',
 'updatedRows': 478,
 'updatedColumns': 10,
 'updatedCells': 4780}

# Playground

In [5]:
# Extract years of experience from description column if contains 'years' followed by 'experience'
df['experience'] = df['description'].apply(lambda x: re.search(r'\d+ years? experience', x) if x != '' else x)

df.head()

,title,company,location,via,date_posted,date_searched,salary,job_type,description,parse_date_posted,posted_on,city,province,min_salary,max_salary,experience
0,Junior Data Analyst,Jiva,"Makassar, Kota Makassar, Sulawesi Selatan",melalui LinkedIn,4 hari yang lalu,2023-11-05,NaN,Pekerjaan tetap,General Overview\n\nEvaluate and analyze poten...,2023-11-01,LinkedIn,Makassar,Sulawesi Selatan,NaN,NaN,[]
1,Data Analyst,Koperasi Simpan Pinjam Sinergi Dana Cipta,"Makassar, Kota Makassar, Sulawesi Selatan",melalui Trabajo.org,3 hari yang lalu,2023-11-05,NaN,Pekerjaan tetap,Persyaratan\n\nAssociate Degree (D3)\n\n1 Year...,2023-11-02,Trabajo.org,Makassar,Sulawesi Selatan,NaN,NaN,[]
2,Data Analyst,PT Tokopedia,"Jakarta, Daerah Khusus Ibukota Jakarta",melalui LinkedIn,2 hari yang lalu,2023-11-05,NaN,Pekerjaan tetap,What You Will Do\n• Become SME (Subject Matter...,2023-11-03,LinkedIn,Jakarta,Daerah Khusus Ibukota Jakarta,NaN,NaN,[]
3,Data Analyst,Kinarya Alihdaya Mandiri PT,"Jakarta, Daerah Khusus Ibukota Jakarta",melalui Trabajo.org,24 jam yang lalu,2023-11-05,NaN,Pekerjaan tetap,Pendidikan Minimal D3 Teknik Informatika/Siste...,2023-11-05,Trabajo.org,Jakarta,Daerah Khusus Ibukota Jakarta,NaN,NaN,[]
4,Data Analyst,Traveloka,"Jakarta, Daerah Khusus Ibukota Jakarta",melalui Cakap Career,5 hari yang lalu,2023-11-05,NaN,Pekerjaan tetap,As a Data Analyst should develop dashboards no...,2023-10-31,Cakap Career,Jakarta,Daerah Khusus Ibukota Jakarta,NaN,NaN,[]


In [6]:
df.experience.value_counts()

experience
[]                                           458
[2 years experience]                           7
[5 years experience]                           6
[5 year experience]                            2
[35 years experience, 3 years experience]      1
[20 years experience, 2 year experience]       1
[2 year experience]                            1
[3 years experience]                           1
Name: count, dtype: int64

In [88]:
# keep only rows with salary_clean not null
isi = df[df['salary_clean'] != '']
isi.salary_clean[6].split('–')[0]

'5500000'

In [ ]:

driver = setup_selenium()
driver.get('https://www.google.com/search?q=data+analyst&ibp=htl;jobs#htivrt=jobs&fpstate=tldetail&htilrad=-1.0&htidocid')

In [ ]:
# Getting job description from right pane
description_pane = driver.find_element(By.CLASS_NAME, 'whazf')

# Try if description is expandable
try:
    # Expand the description
    description_pane.find_element(By.CLASS_NAME, 'mjkhcd').click()
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'config-text-expandable')))
    description = description_pane.find_element(By.CLASS_NAME, 'config-text-expandable').text
except NoSuchElementException:
    description = description_pane.find_element(By.CLASS_NAME, 'HBvzbc').text

print(description)